<a href="https://colab.research.google.com/github/chidinma-godwin/course-recommendation-nlp/blob/main/course_reccomendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 4.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
# Suppress the TqdmExperimentalWarning
from tqdm import TqdmExperimentalWarning
import warnings
warnings.filterwarnings('ignore', category=TqdmExperimentalWarning)

In [6]:
import re
import string
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from gensim.models import Word2Vec, Doc2Vec
from gensim.models.doc2vec import TaggedDocument

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers import SentenceTransformer

## Data Exploration

In [26]:
base_dir = "/content/drive/MyDrive/project"

df = pd.read_csv(f"{base_dir}/courseslist.csv")
df.head()

,additional_course_detail,course_avg_rating,course_certificate,course_institution,course_is_classroom,course_is_university,course_level,course_link,course_name,course_num_rating,course_provider,course_subject,course_type,description,duration,is_all_time_best,pricing,start_date,teacher
35158,"Livestream begins,Welcome to the Python Pulse,...",0.0,False,Visual Studio Code,True,False,NaN,https://www.classcentral.com/classroom/youtube...,Jupyter Notebooks in VS Code,0,YouTube,Visual Studio Code,course,Master MicroPython with Kevin McAleer in under...,21 minutes,False,Free Online Course,On-Demand,NaN
35159,"| Video Overview,| Project Overview,| Project ...",0.0,False,Tech with Tim,True,False,NaN,https://www.classcentral.com/classroom/youtube...,Learn JavaScript With This One Project,0,YouTube,Javascript,course,Master MicroPython with Kevin McAleer in under...,21 minutes,False,Free Online Course,On-Demand,NaN
35160,"Layout,Navigation bar,Hero Section,About Secti...",0.0,False,JavaScript Mastery,True,False,NaN,https://www.classcentral.com/classroom/youtube...,Build and Deploy an Amazing 3D Web Developer P...,0,YouTube,React,course,Master MicroPython with Kevin McAleer in under...,21 minutes,False,Free Online Course,On-Demand,NaN
35161,Learning Crystal Language programming - part 1...,0.0,False,NaN,True,False,NaN,https://www.classcentral.com/classroom/youtube...,Crystal,0,YouTube,Crystal,course,Master MicroPython with Kevin McAleer in under...,21 minutes,False,Free Online Course,On-Demand,NaN
35162,"Intro,Welcome,Next.js Update,Create New Projec...",0.0,False,Dave Gray,True,False,NaN,https://www.classcentral.com/classroom/youtube...,"Next.js Data Fetching, Dynamic Routes & Metada...",0,YouTube,Next.js,course,Master MicroPython with Kevin McAleer in under...,21 minutes,False,Free Online Course,On-Demand,NaN


In [21]:
df.duplicated().sum()

0

In [27]:
df = df.replace(np.nan, None)
df.isna().sum()

additional_course_detail        0
course_avg_rating               0
course_certificate              0
course_institution          20813
course_is_classroom             0
course_is_university            0
course_level                16715
course_link                     0
course_name                     0
course_num_rating               0
course_provider                 0
course_subject                  0
course_type                     0
description                     0
duration                       65
is_all_time_best                0
pricing                         0
start_date                      0
teacher                     10840
dtype: int64

In [22]:
course_subjects_df = pd.DataFrame(df.groupby('course_subject').size(), columns=["count"])
course_subjects_df = course_subjects_df.sort_values(by='count', ascending=False)
course_subjects_df

,count
course_subject,
Python,1332
Microsoft Azure,1198
Google Cloud Platform (GCP),1111
Uncategorized,1008
Javascript,812
...,...
AWS Artifact,1
Set Theory,1
Tricentis Tosca,1


In [24]:
course_subjects_df = pd.DataFrame(df.groupby('course_level', dropna=False).size(), columns=["count"])
course_subjects_df = course_subjects_df.sort_values(by='count', ascending=False)
course_subjects_df

,count
course_level,
NaN,16715
beginner,11844
intermediate,5625
advanced,979


In [28]:
course_subjects_df = pd.DataFrame(df.groupby('pricing').size(), columns=["count"])
course_subjects_df = course_subjects_df.sort_values(by='count', ascending=False)
course_subjects_df

,count
pricing,
Free Online Course,11498
Free Trial Available,9615
Paid Course,9502
Free Online Course (Audit),2987
Conference Talk,747
Free Certificate,525
$14.00,28
$518.00,15
$79.00,15


In [13]:
course_subjects_df = pd.DataFrame(df.groupby('course_is_university').size(), columns=["count"])
course_subjects_df = course_subjects_df.sort_values(by='count', ascending=False)
course_subjects_df

,count
course_is_university,
False,32392
True,2771


In [29]:
course_subjects_df = pd.DataFrame(df.groupby('course_provider').size(), columns=["count"])
course_subjects_df = course_subjects_df.sort_values(by='count', ascending=False)
course_subjects_df

,count
course_provider,
YouTube,8452
Udemy,8248
Pluralsight,4389
Coursera,3711
LinkedIn Learning,3620
...,...
Federica,1
Stepik,1
Semrush Academy,1


In [30]:
df[df['course_subject'] == 'Uncategorized'].head()

,additional_course_detail,course_avg_rating,course_certificate,course_institution,course_is_classroom,course_is_university,course_level,course_link,course_name,course_num_rating,course_provider,course_subject,course_type,description,duration,is_all_time_best,pricing,start_date,teacher
478,"Introduction,Project Overview,Case Scenario,Al...",0.0,False,Stanford University,True,True,None,https://www.classcentral.com/classroom/youtube...,Picking on the Same Person - Does Algorithmic ...,0,YouTube,Uncategorized,course,Stanford University offers a brief seminar exp...,35 minutes,False,Free Online Course,On-Demand,None
479,"Introduction,Linear approach,landscape changes...",0.0,False,Stanford University,True,True,None,https://www.classcentral.com/classroom/youtube...,"AI, Archaeology, and Archives - How Data Scien...",0,YouTube,Uncategorized,course,Stanford University offers a brief seminar exp...,35 minutes,False,Free Online Course,On-Demand,None
480,"Introduction,Our Story Begins,The History of t...",0.0,False,Stanford University,True,True,None,https://www.classcentral.com/classroom/youtube...,"Voices in the Code - A Story About People, The...",0,YouTube,Uncategorized,course,Stanford University offers a brief seminar exp...,35 minutes,False,Free Online Course,On-Demand,None
497,Stanford Seminar: PyWren - Pushing Microservic...,0.0,False,Stanford University,True,True,None,https://www.classcentral.com/classroom/youtube...,Stanford Seminar - PyWren - Pushing Microservi...,0,YouTube,Uncategorized,course,Stanford University offers a brief seminar on ...,45 minutes,False,Free Online Course,On-Demand,None
498,"Stanford Seminar: Data For The People, Andreas...",0.0,False,Stanford University,True,True,None,https://www.classcentral.com/classroom/youtube...,Data for the People - Andreas Weigend of Socia...,0,YouTube,Uncategorized,course,Stanford University offers a brief seminar on ...,45 minutes,False,Free Online Course,On-Demand,None
